# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

## Question 1: What is the sum of the outputs of the generator for limit = 5?

`sum = 8.382332347441762`

### Answer 1: `C: 8.382`

- **A**: 10.23433234744176
- **B**: 7.892332347441762
- **C**: 8.382332347441762
- **D**: 9.123332347441762

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13 # qn1: 5, qn2: 13
generator = square_root_generator(limit)

sum = 0
values = []
for sqrt_value in generator:
    print(sqrt_value)
    # sum += sqrt_value
    values.append(sqrt_value)

# print(f'sum = {sum}')
print(f'13th item: {values[-1]}')

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
13th item: 3.605551275463989


## Question 2: What is the 13th number yielded

`13th item: 3.605551275463989`

### Answer 2: `B: 3.6055`

- **A**: 4.236551275463989
- **B**: 3.605551275463989
- **C**: 2.345551275463989
- **D**: 5.678551275463989

Change limit = 13

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [3]:
import dlt
import duckdb

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
							table_name="persons",
							write_disposition="replace",
							primary_key="ID")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)



Pipeline dlt_ipykernel_launcher load step completed in 0.34 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////mnt/c/Users/ellabelle/Github/data-engineering-zoomcamp/cohorts/2024/workshops/dlt_resources/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707495152.6111448 is LOADED and contains no failed jobs


In [4]:
# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(people_2(),
							table_name="persons",
							write_disposition="append",
							primary_key="ID")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.31 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////mnt/c/Users/ellabelle/Github/data-engineering-zoomcamp/cohorts/2024/workshops/dlt_resources/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707495153.3081484 is LOADED and contains no failed jobs


In [5]:
# show outcome

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n table below:")

people = conn.sql("SELECT * FROM persons").df()
display(people)

# As you can see, the same data was loaded in both cases.

total_age = conn.sql("SELECT SUM(age) FROM persons")
print(total_age)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ persons             │
└─────────────────────┘




 table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707495152.6111448,GP5hHe1Cbr+3GA,None
1,2,Person_2,27,City_A,1707495152.6111448,4uLyxhBTDIAsXg,None
2,3,Person_3,28,City_A,1707495152.6111448,aGnW4S6xf0fC9g,None
3,4,Person_4,29,City_A,1707495152.6111448,Qp8hQ/MvQ8drPw,None
4,5,Person_5,30,City_A,1707495152.6111448,IJoV9Si1cuNpGw,None
5,3,Person_3,33,City_B,1707495153.3081484,gt5t7wH39RVlZQ,Job_3
6,4,Person_4,34,City_B,1707495153.3081484,vci+Lg9XqNmjQg,Job_4
7,5,Person_5,35,City_B,1707495153.3081484,MNQRlIE28jV8CQ,Job_5
8,6,Person_6,36,City_B,1707495153.3081484,MsL1e9l42tkiWA,Job_6
9,7,Person_7,37,City_B,1707495153.3081484,oH0J+fRDhaWplQ,Job_7


┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘



# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


## Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.

### Answer 3: `A: 353`

- **A**: 353
- **B**: 365
- **C**: 378
- **D**: 390

## Question 4: Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

### Answer 4: `B: 266`

- **A**: 215
- **B**: 266
- **C**: 241
- **D**: 258

# Solution: First make sure that the following modules are installed:

In [6]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

UsageError: Line magic function `%%capture` not found.


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX